# Подготовка данных

### Импорт библиотек

In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Рестораны

In [3]:
df_restaurants = pd.read_json('../mining/restaurants.json')

In [4]:
df_restaurants.shape

(14041, 12)

In [5]:
df_restaurants.head()

,url,title,rating,review_count,street_adress,work_time,geodata,full_data,price,locality,assessments_info,phone_number
0,https://www.tripadvisor.ru//Restaurant_Review-...,Любовь Пирогова,"5,0",138 отзыва,"ул. Мытная, 74 Даниловский рынок, Москва 11519...",Закрыто: Посмотреть время работы,"Местоположение и контактные данныеул. Мытная,...","ДИАПАЗОН ЦЕН150 руб - 500 рубТИП КУХНИРусская,...",150 руб - 500 руб,"Русская, Кафе","{'5_star': '122', '4_star': '4', '3_star': '3'...",+7 495 120-00-97
1,https://www.tripadvisor.ru//Restaurant_Review-...,Джумбус,"5,0",320 отзыва,"ул. Добровольческая 12, Москва 109004 Россия",Открыто: 12:00 PM - 11:00 PM,Местоположение и контактные данныеул. Доброво...,ДИАПАЗОН ЦЕН100 руб - 1 000 рубТИП КУХНИСредиз...,100 руб - 1 000 руб,"Средиземноморская, Барбекю, Гриль, Восточноевр...","{'5_star': '279', '4_star': '11', '3_star': '9...",+7 903 590-34-70
2,https://www.tripadvisor.ru//Restaurant_Review-...,[KU:] рамен изакая бар,"5,0",143 отзыва,"Большая Грузинская, 69, Москва 123056 Россия",Открыто: 11:00 AM - 12:00 AM,Местоположение и контактные данныеБольшая Гру...,ДИАПАЗОН ЦЕН240 руб - 880 рубТИП КУХНИЯпонская...,240 руб - 880 руб,"Японская, Бар, Азиатская, Гастропаб, Супы","{'5_star': '131', '4_star': '2', '3_star': '3'...",+7 499 609-00-69
3,https://www.tripadvisor.ru//Restaurant_Review-...,АндерСон,"5,0",222 отзыва,"Жулебинский бульвар, 5 ТЦ ""Динамит"", Москва 10...",Открыто: 10:30 AM - 11:30 PM,Местоположение и контактные данныеЖулебинский...,ДИАПАЗОН ЦЕН300 руб - 1 500 рубТИП КУХНИЕвропе...,300 руб - 1 500 руб,"Европейская, Современная, Центральноевропейская","{'5_star': '191', '4_star': '14', '3_star': '5...",+7 495 252-85-75
4,https://www.tripadvisor.ru//Restaurant_Review-...,Сабор де ла Вида Ресторан,"5,0",323 отзыва,"ул. 1905 года, 10/1, Москва 123022 Россия",Открыто: 12:00 PM - 11:45 PM,Местоположение и контактные данныеул. 1905 го...,ДИАПАЗОН ЦЕН280 руб - 2 500 рубТИП КУХНИСредиз...,280 руб - 2 500 руб,Средиземноморская,"{'5_star': '275', '4_star': '16', '3_star': '7...",+7 926 222-71-22


In [6]:
# df_restaurants['work_time'] = df_restaurants['work_time'].str.encode('ascii', 'ignore').str.decode('utf-8').str.replace(',', '').\
#     str.replace(' ', '').str.split('-').apply(lambda x: [w.strip() for w in x])

In [7]:
# df_restaurants['work_time_min'] = df_restaurants['work_time'].apply(lambda x: x[0])
# df_restaurants['work_time_max'] = df_restaurants['work_time'].apply(lambda x: x[1])

In [8]:
for i in range(1, 6):
    column_name = '{}_star'.format(i)
    df_restaurants[column_name] = df_restaurants['assessments_info'].apply(lambda x: x.get(column_name)).\
        str.encode('ascii', 'ignore').str.decode('utf-8').astype(float).fillna(0).astype(int)

In [9]:
df_restaurants['review_count'] = df_restaurants['review_count'].str.encode('ascii', 'ignore').str.decode('utf-8').replace('', '0').astype(int)

In [10]:
df_restaurants['rating'] = df_restaurants['rating'].str.encode('ascii', 'ignore').str.decode('utf-8').str.replace(',', '.').replace('', '0').astype(float)

In [11]:
stop_words = [
    'Завтрак',
    'Завтрак типа "шведский стол"',
    'Обед', 
    'Ужин', 
    'Принимаются только наличные', 
    'Утвержденная парковка', 
    'Электронные платежи',
    'Бесплатный Wi-Fi', 
    'Места для сидения',
    'Обслуживание посетителей за\xa0столиками',
    'Принимаются карты Mastercard',
    'Принимаются карты Visa',
    'Для посетителей на автомобилях',
    'Имеется парковка', 
    'Парковка на улице', 
    'Телевизор',
    'Принимаются кредитные карты',
    'Для\xa0гостей с\xa0собаками',
    'Открыто допоздна',
    '',
]

In [12]:
df_restaurants['kitchen'] = df_restaurants['locality'].str.split(',').apply(lambda x: [w.strip() for w in x]).\
    apply(lambda x: [w for w in x if w not in stop_words]).apply(lambda x: [w for w in x if 'руб' not in w])

In [13]:
df_restaurants['kitchen'].explode().unique()

array(['Русская', 'Кафе', 'Средиземноморская', 'Барбекю', 'Гриль',
       'Восточноевропейская', 'Европейская', 'Центральноевропейская',
       'Японская', 'Бар', 'Азиатская', 'Гастропаб', 'Супы', 'Современная',
       'Французская', 'Испанская', 'Паб', 'Итальянская', 'Пицца',
       'Морепродукты', 'Бельгийская', 'Стейк-хаус',
       'Подходит для вегетарианцев', 'Для веганов', 'Безглютеновые блюда',
       'Напитки', 'Американская', 'Грузинская', 'Фастфуд', 'Фьюжн',
       'Винный бар', 'Кавказская', 'Аргентинская', 'Ирландская',
       'Британская', 'Шотландская', 'Украинская', 'Здоровая',
       'Международная', 'Тосканская', 'Центрально-итальянская',
       'Каталанская', 'Неаполитанская', 'Кухня Кампании', 'Суши',
       'Южно-итальянская', 'Шведская', 'Скандинавская', 'Датская',
       'Норвежская', nan, 'Закусочная', 'Кошерная', 'Немецкая',
       'Корейская', 'Карибская', 'Гавайская', 'Ямайская', 'Бранч',
       'Филиппинская', 'Кухня Хоккайдо', 'Ближневосточная', 'Китайская',

In [14]:
df_restaurants['price'] = df_restaurants['price'].str.encode('ascii', 'ignore').str.decode('utf-8').str.replace(',', '').\
    str.replace(' ', '').str.split('-').apply(lambda x: [w.strip() for w in x])

In [15]:
df_restaurants['price_min'] = df_restaurants['price'].apply(lambda x: int(x[0]) if x[0] != '' else 0)
df_restaurants['price_max'] = df_restaurants['price'].apply(lambda x: int(x[1]) if x[0] != '' else 0)

In [16]:
df_restaurants['geodata'][0]

' Местоположение и контактные данныеул. Мытная, 74 Даниловский рынок, Москва 115191 РоссияДаниловскийСайтЭлектронная почта+7 495 120-00-97Исправить данные об объекте'

In [17]:
restaurants = df_restaurants['title'].unique().tolist()

In [18]:
# restaurants

In [19]:
df_restaurants['rest_id'] = df_restaurants['url'][0].split('-')[1] + '-' + df_restaurants['url'][0].split('-')[2]

In [20]:
columns = [
    'rest_id',
    'url',
    'title',
    'rating',
    'review_count',
    'street_adress',
    'phone_number',
    '1_star',
    '2_star',
    '3_star',
    '4_star',
    '5_star',
    'kitchen',
    'price_min',
    'price_max',
    'work_time',
]

In [21]:
df_restaurants[columns].head()

,rest_id,url,title,rating,review_count,street_adress,phone_number,1_star,2_star,3_star,4_star,5_star,kitchen,price_min,price_max,work_time
0,g298484-d10841351,https://www.tripadvisor.ru//Restaurant_Review-...,Любовь Пирогова,5.0,138,"ул. Мытная, 74 Даниловский рынок, Москва 11519...",+7 495 120-00-97,2,3,3,4,122,"[Русская, Кафе]",150,500,Закрыто: Посмотреть время работы
1,g298484-d10841351,https://www.tripadvisor.ru//Restaurant_Review-...,Джумбус,5.0,320,"ул. Добровольческая 12, Москва 109004 Россия",+7 903 590-34-70,4,2,9,11,279,"[Средиземноморская, Барбекю, Гриль, Восточноев...",100,1000,Открыто: 12:00 PM - 11:00 PM
2,g298484-d10841351,https://www.tripadvisor.ru//Restaurant_Review-...,[KU:] рамен изакая бар,5.0,143,"Большая Грузинская, 69, Москва 123056 Россия",+7 499 609-00-69,3,2,3,2,131,"[Японская, Бар, Азиатская, Гастропаб, Супы]",240,880,Открыто: 11:00 AM - 12:00 AM
3,g298484-d10841351,https://www.tripadvisor.ru//Restaurant_Review-...,АндерСон,5.0,222,"Жулебинский бульвар, 5 ТЦ ""Динамит"", Москва 10...",+7 495 252-85-75,5,3,5,14,191,"[Европейская, Современная, Центральноевропейская]",300,1500,Открыто: 10:30 AM - 11:30 PM
4,g298484-d10841351,https://www.tripadvisor.ru//Restaurant_Review-...,Сабор де ла Вида Ресторан,5.0,323,"ул. 1905 года, 10/1, Москва 123022 Россия",+7 926 222-71-22,2,2,7,16,275,[Средиземноморская],280,2500,Открыто: 12:00 PM - 11:45 PM


In [22]:
df_restaurants[columns].to_feather('../data/restaurants.feather')

# Обзоры

In [3]:
df_reviews = pd.read_json('../mining/reviews.json')

In [4]:
df_reviews.shape

(332139, 8)

In [5]:
df_reviews['rest_id'] = df_reviews['url'][0].split('-')[1] + '-' + df_reviews['url'][0].split('-')[2]

In [6]:
df_reviews['review_id'] = df_reviews['url'][0].split('-')[1] + '-' + df_reviews['url'][0].split('-')[2] + '-' + df_reviews['url'][0].split('-')[3]

In [7]:
rev_columns = [
    'rest_id',
    'review_id',
    'restaurant_name',
    'url',
    'username',
    'date',
    'review_title',
    'review_body',
    'date_of_visit',
    'stars_amount',
]

In [8]:
df_reviews[rev_columns].head()

,rest_id,review_id,restaurant_name,url,username,date,review_title,review_body,date_of_visit,stars_amount
0,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,olgusik1995,Отзыв написан неделю назад,Вкусный визит,Вкусно) очень вкусно попробовали несколько раз...,Дата посещения: декабрь 2021 г.,5
1,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,alexsW1908FJ,Отзыв написан неделю назад,Необычные сочетания продуктов в начинке,Необычные сочетания продуктов в начинке. Безус...,Дата посещения: декабрь 2021 г.,5
2,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,_K6782NJ,Отзыв написан неделю назад,Лучшие пироги тут,Впервые были тут и попробовали пироги. Нежнейш...,Дата посещения: декабрь 2021 г.,5
3,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,Alexey_Kalinin,Отзыв написан 2 недели назад,Лучшие пироги в Москве,Домой с доставкой заказали. Как всегда идеальн...,Дата посещения: декабрь 2021 г.,5
4,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,803evgeniias,Отзыв написан 2 недели назад,Доставка наивкуснейших пирогов,Без притворства это наивкуснейшие пироги! Зака...,Дата посещения: декабрь 2021 г.,5


In [9]:
df_reviews[rev_columns].to_feather('../data/reviews.feather')